# Lab Practice: Online Learning

In [1]:
import pandas as pd

# Batch Libraries (Scikit-Learn)
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

# Online Libraries (River)
from river import compose, tree, metrics, preprocessing

# Utilities
from lab1_utils import *

# Configuration
DATA_DIR = 'data/'

## Data loading and target preparation

In [2]:
# Load dataset
file_path = DATA_DIR + 'spy.us.txt'
df = pd.read_csv(file_path)

# Delete 'OpenInt' column (always 0 value)
if 'OpenInt' in df.columns:
    df = df.drop(columns=['OpenInt'])
    print("'OpenInt' column deleted successfully")

print(f"Initial number of rows: {len(df)}")

# Delete rows with missing values
df_cleaned = df.dropna()

print(f"Final columns: {df_cleaned.columns.tolist()}")
print(f"Final number of rows: {len(df_cleaned)}")

# Save cleaned dataset
df_cleaned.to_csv(DATA_DIR + 'spy_cleaned.csv', index=False)

'OpenInt' column deleted successfully
Initial number of rows: 3201
Final columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
Final number of rows: 3201


In [3]:
# Load cleaned dataset
df = pd.read_csv(DATA_DIR + 'spy_cleaned.csv')

# Create the target: the 'High' 5 records ahead
# shift(-5) moves the values up by 5 positions.
# We are now predicting the value 5 days into the future to avoid naive correlation.
df['Target_High'] = df['High'].shift(-5)

# The last 5 rows do not have a "5th day future," so they become null (NaN)
df = df.dropna(subset=['Target_High'])

# Verify that Target_High is indeed the next day's High
print("\nFirst rows of the transformed dataset:")
df[['Date', 'High', 'Target_High']].head(21)


First rows of the transformed dataset:


,Date,High,Target_High
0,2005-02-25,106.00,107.01
1,2005-02-28,105.68,107.38
2,2005-03-01,105.87,107.15
3,2005-03-02,106.22,106.52
4,2005-03-03,106.20,105.85
5,2005-03-04,107.01,106.04
6,2005-03-07,107.38,105.56
7,2005-03-08,107.15,105.82
8,2005-03-09,106.52,104.44
9,2005-03-10,105.85,104.31


## Batch Learning

### Feature extraction and data splitting

In [4]:
# Data Splitting (Temporal Split - NO SHUFFLE): 70% training, 15% validation, and 15% test
n = len(df)
train_end = int(n * 0.70)
val_end = int(n * 0.85)

print(f"Total samples: {n}")

# Generate features and target for the entire dataset (train + val + test) to ensure we have the necessary history for feature generation
# This prevents the first 20 rows of the validation or test sets from becoming NaN due to insufficient historical data
X_full, y_full = get_batch_features_and_target(df)
print(f"Full shapes: X={X_full.shape}, y={y_full.shape}")

# Data splitting based on indices making sure that the cut respects the reduced dataframe after feature generation
X_train = X_full.loc[X_full.index < train_end]
y_train = y_full.loc[y_full.index < train_end]

X_val = X_full.loc[(X_full.index >= train_end) & (X_full.index < val_end)]
y_val = y_full.loc[(y_full.index >= train_end) & (y_full.index < val_end)]

X_test = X_full.loc[X_full.index >= val_end]
y_test = y_full.loc[y_full.index >= val_end]

print(f"Train shapes: X={X_train.shape}, y={y_train.shape}")
print(f"Val shapes: X={X_val.shape}, y={y_val.shape}")
print(f"Test shapes: X={X_test.shape}, y={y_test.shape}")

Total samples: 3196
Full shapes: X=(3176, 8), y=(3176,)
Train shapes: X=(2217, 8), y=(2217,)
Val shapes: X=(479, 8), y=(479,)
Test shapes: X=(480, 8), y=(480,)


In [5]:
X_train.head()

,Close,Return,Mean_return_5,Mean_return_20,Volatility_20,Open_Close_Diff,Volume,Volume_dev
20,102.19,0.001372,-0.002071,-0.001712,0.005964,-0.10,53885043,-15613298.30
21,101.51,-0.006654,-0.002686,-0.001709,0.005962,-0.54,83849903,14143833.75
22,102.96,0.014284,0.002193,-0.001252,0.006766,1.11,72199510,1613835.85
23,102.77,-0.001845,0.001667,-0.001320,0.006765,-0.20,74669192,4036712.75
24,102.30,-0.004573,0.000517,-0.001568,0.006789,-1.04,111800673,39142501.25


In [6]:
y_train.head()

20    102.67
21    103.12
22    103.62
23    103.89
24    103.85
Name: Target_High, dtype: float64

In [7]:
# Scale the data (standarization) to avoid inestability
scaler = StandardScaler()
# Obtain mean and std from the training data only to prevent data leakage
scaler.fit(X_train)

# Transfrom all sets with the obtained parameters from training set
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print("Shapes after scaling:")
print(f"Train shapes: X={X_train_scaled.shape}, y={y_train.shape}")
print(f"Val shapes: X={X_val_scaled.shape}, y={y_val.shape}")
print(f"Test shapes: X={X_test_scaled.shape}, y={y_test.shape}")

Shapes after scaling:
Train shapes: X=(2217, 8), y=(2217,)
Val shapes: X=(479, 8), y=(479,)
Test shapes: X=(480, 8), y=(480,)


### Model 1: RandomForest

In [8]:
# Define and train model
print("Training Random Forest...")
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train_scaled, y_train)

Training Random Forest...


,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [9]:
# Evaluate model
mae_rf_val = mean_absolute_error(y_val, rf_model.predict(X_val_scaled))
print(f"Random Forest -> Val MAE: {mae_rf_val:.4f}")

Random Forest -> Val MAE: 19.6112


### Model 2: GradientBoosting

In [10]:
# Define and train model
print("Training Gradient Boosting...")
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
gb_model.fit(X_train_scaled, y_train)

Training Gradient Boosting...


,loss,'squared_error'
,learning_rate,0.1
,n_estimators,100
,subsample,1.0
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,3
,min_impurity_decrease,0.0
,init,None


In [11]:
# Evaluation GBM
mae_gb_val = mean_absolute_error(y_val, gb_model.predict(X_val_scaled))
print(f"Gradient Boost -> Val MAE: {mae_gb_val:.4f}")

Gradient Boost -> Val MAE: 19.7642


## Online Learning

In [12]:
# Create Pipeline: Feature Extraction -> Model
# We use HoeffdingAdaptiveTreeRegressor
river_model = compose.Pipeline(
    OnlineSPYFeatures(),   # Custom transformer to extract features from the raw data stream
    preprocessing.StandardScaler(),  # Scales the data (standarization) to avoid inestability
    tree.HoeffdingAdaptiveTreeRegressor(
        grace_period=50,       # How often to check for splits
        model_selector_decay=0.9
    )
)

# Metric for Validation
metric_val = metrics.MAE()

In [13]:
# We iterate through the entire dataset sequentially
print("Processing stream row by row...")

for i, (index, row) in enumerate(df.iterrows()):
    
    # Prepare input (x) and target (y)
    x = row[['Open', 'High', 'Low', 'Close', 'Volume']].to_dict()
    y = row['Target_High']
    
    # 1. PREDICT (Test-then-Train approach)
    # We make a prediction before the model learns from this specific example
    y_pred = river_model.predict_one(x)   # predict_one takes the x, transforms it with OnlineSPYFeatures and StandardScaler, and predicts with the model
    
    # 2. UPDATE METRICS
    # We only update the metric if we are currently in the Validation segment (would be equivalent for test)
    if index in X_val.index:
        metric_val.update(y, y_pred)
        
    # 3. LEARN
    # The model updates its internal state with the new truth
    river_model.learn_one(x, y)

Processing stream row by row...


In [14]:
print(f"Online HoeffdingAdaptiveTreeRegressor -> Val MAE: {metric_val.get():.4f}")

Online HoeffdingAdaptiveTreeRegressor -> Val MAE: 4.1314
